In [2]:
import pandas as pd
import re

rmovies = pd.read_csv('data/movies_data/ratings_movies.csv')

In [3]:
rmovies

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [4]:
def get_year_release(arg):
    candidates = re.findall(r'\(\d{4}\)', arg)
    if len(candidates) > 0:
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else: 
        return None
    
rmovies['year_release'] = rmovies['title'].apply(get_year_release)
# print(rmovies['year_release'].shape[0] - rmovies['year_release'].count())

mask = rmovies['year_release'] == 1999
rmovies[mask].groupby(by='title')['rating'].mean().sort_values()





title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [5]:
mask_genre = rmovies['year_release'] == 2010.0
rmovies[mask_genre].groupby(by='genres')['rating'].mean().sort_values()

rmovies.groupby(by='userId')['genres'].nunique().sort_values(ascending=False)

rmovies.groupby(by='userId')['rating'].agg(
    ['count', 'mean']
).sort_values(by=['count', 'mean'], ascending=[True, False])


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [6]:
mask3 = rmovies['year_release'] == 2018
rmovies[mask3].groupby(by='genres')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[False, False])

,count,mean
genres,,
Action|Adventure|Sci-Fi,14,3.928571
Action|Comedy|Sci-Fi,12,3.875000
Comedy,8,3.312500
Action|Adventure|Children|Sci-Fi,5,3.900000
Animation|Comedy,5,3.500000
Action|Adventure|Animation|Children,4,3.000000
Drama|Horror|Thriller,4,2.750000
Action|Adventure|Fantasy,4,2.500000
Adventure|Mystery|Sci-Fi|Thriller,3,3.833333


In [7]:

rmovies['year_rating'] = pd.to_datetime(rmovies['date'])
rmovies['year_rating'] = rmovies['year_rating'].dt.year
pivot = rmovies.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc='mean',
    fill_value=0
)

print(pivot)

year_rating                              1996      1997  1998  1999      2000  \
genres                                                                          
(no genres listed)                   0.000000  0.000000   0.0   0.0  0.000000   
Action                               2.730769  3.538462   0.0   0.0  2.588235   
Action|Adventure                     3.454545  4.150000   4.2   4.0  3.738462   
Action|Adventure|Animation           0.000000  0.000000   0.0   0.0  0.000000   
Action|Adventure|Animation|Children  0.000000  0.000000   0.0   0.0  0.000000   
...                                       ...       ...   ...   ...       ...   
Sci-Fi|Thriller                      2.666667  3.400000   0.0   4.0  2.142857   
Sci-Fi|Thriller|IMAX                 0.000000  0.000000   0.0   0.0  0.000000   
Thriller                             3.838095  3.923077   3.8   3.7  3.087912   
War                                  0.000000  0.000000   0.0   4.5  3.000000   
Western                     

In [8]:
orders = pd.read_csv('data/internet-store/orders.csv', sep=';')
products = pd.read_csv('data/internet-store/products.csv', sep=';')

orders_products = orders.merge(
    products,
    left_on='ID товара',
    right_on='Product_ID',
    how='left'
).drop('Product_ID', axis=1)

orders_products

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,"Носки Подарочные, жен",249.0,RUR


In [9]:
orders_products[orders_products['Статус'] == 'Отменён']

orders_products.groupby(by='ID Покупателя')['Price'].sum()

ID Покупателя
1       448.0
5     14042.0
7     17096.0
8       697.0
9       344.0
10      199.0
Name: Price, dtype: float64

In [10]:
df1 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005', 'id006', 'id007'],
                    'first_name': ['Rivi', 'Wynnie', 'Kristos', 'Madalyn', 'Tobe', 'Regan', 'Kristin'],
                    'last_name': ['Valti', 'McMurty', 'Ivanets', 'Max', 'Riddich', 'Huyghe', 'Illis'],
                    'email': ['[email protected]', '[email protected]', '[email protected]',
                              '[email protected]', '[email protected]', '[email protected]', '[email protected]']
                    })


df2 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005'],
                    'image_url': ['http://example.com/img/id001.png', 'http://example.com/img/id002.jpg',
                                  'http://example.com/img/id003.bmp', 'http://example.com/img/id004.jpg',
                                  'http://example.com/img/id005.png']
                    })

df3_merged = pd.merge(df1, df2, how='outer', indicator=True)
df3_merged


,user_id,first_name,last_name,email,image_url,_merge
0,id001,Rivi,Valti,[email protected],http://example.com/img/id001.png,both
1,id002,Wynnie,McMurty,[email protected],http://example.com/img/id002.jpg,both
2,id003,Kristos,Ivanets,[email protected],http://example.com/img/id003.bmp,both
3,id004,Madalyn,Max,[email protected],http://example.com/img/id004.jpg,both
4,id005,Tobe,Riddich,[email protected],http://example.com/img/id005.png,both
5,id006,Regan,Huyghe,[email protected],NaN,left_only
6,id007,Kristin,Illis,[email protected],NaN,left_only
